In [1]:
import pickle5 as pickle
import random
import numpy as np

with open('../tokens', 'rb') as pickle_file:
    tokenized_sent = pickle.load(pickle_file)

random.shuffle(tokenized_sent)

def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

test_tokens = tokenized_sent[:10000]
train_tokens = [' '.join(token[2]) for token in tokenized_sent[10000:]]

In [2]:
!pip install sentence-transformers

Defaulting to user installation because normal site-packages is not writeable


In [3]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')

/home/jk/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
sentence_embeddings = model.encode(train_tokens)

In [8]:
query = ''.join(test_tokens[0][2])
query_vec = model.encode([query])[0]

In [11]:
for sent in train_tokens[:10]:
  sim = cosine(query_vec, model.encode([sent])[0])
  print("Sentence = ", sent, "; similarity = ", sim)

Sentence =  we provide evidence that democracy has a positive effect on gdp per capita our dynamic panel strategy controls for country fixed effects and the rich dynamics of gdp which otherwise confound the effect of democracy to reduce measurement error we introduce a new indicator of democracy that consolidates previous measures our baseline results show that democratizations increase gdp per capita by about 20 percent in the long run we find similar effects using a propensity score reweighting strategy as well as an strategy using regional waves of democratization the effects are similar across different levels of development and appear to be driven by greater investments in capital schooling and health ; similarity =  -0.049868003
Sentence =  lane detection the process of identifying lane markings as approximated curves is widely used for lane departure warning and adaptive cruise control in autonomous vehicles the popular pipeline that solves it in two feature extraction plus whil